In [5]:
import string
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
import string 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/williamchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read Dataset

In [6]:
imdb_data = pd.read_csv('./IMDB_Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### Split the data

In [193]:
train_reviews=imdb_data.review[:30000]
train_sentiments=imdb_data.sentiment[:30000]

val_reviews=imdb_data.review[30000:40000]
val_sentiments=imdb_data.sentiment[30000:40000]

test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]

## Preprocess Dataset

In [194]:
def preprocess(reviews):
    container = []
    for review in reviews:
        review = review.replace("<br />", "")
        for ele in string.punctuation:
                if ele in review:
                        review = review.replace(ele, "")
        container.append(review)
    return container

def preprocess_sentiment(sentiments):
    container = []
    for sentiment in sentiments:
        container.append(sentiment)
    return container

In [195]:
train_reviews = preprocess(train_reviews)
val_reviews = preprocess(val_reviews)
test_reviews = preprocess(test_reviews)
train_sentiments = preprocess_sentiment(train_sentiments)
val_sentiments = preprocess_sentiment(val_sentiments)
test_sentiments = preprocess_sentiment(test_sentiments)

In [196]:
'''
Method that take in the training dataset, then return the positive and negative words log probability.
Input: train_reviews: reviews (sentences) for training
       train_sentiments: sentiments (label) for training
       val_reviews: reviews (sentences) for validation
       test_reviews: reviews (sentences) for testing
       tfidf: boolean variable indicating whether using bow or tfidf
       alpha: laplance smoothing variable, default to be 1.0
       ngram_range: the scale of ngram model will be used, default = (1,1) unigram
return: negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class
        positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class
        mnb: the trained multinomial naive bayes model, later can be used for testing
        transformed_val_reviews: transformed val reviews that later can be used for validation
        transformed_test_reviews: transformed test reviews that later can be used for testing
        vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
'''

def generate_log_prob(train_reviews, train_sentiments,  test_reviews, val_reviews=None, tfidf=False, alpha=1.0, ngram_range = (1,1)):

    if (tfidf):
        #Tfidf vectorizer
        vec=TfidfVectorizer(use_idf=tfidf, ngram_range=ngram_range)
        transformed_train_reviews=vec.fit_transform(train_reviews)
        if val_reviews is not None:
            transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)
    else:
        vec=CountVectorizer(ngram_range=(1,1))
        transformed_train_reviews=vec.fit_transform(train_reviews)
        if val_reviews is not None:
            transformed_val_reviews = vec.transform(val_reviews)
        transformed_test_reviews=vec.transform(test_reviews)

    #training the model
    mnb = MultinomialNB(alpha=alpha)

    #fitting the naive bayes for bag of words
    mnb = mnb.fit(transformed_train_reviews, train_sentiments)
    
    negative_log_prob = mnb.feature_log_prob_[0]
    positive_log_prob = mnb.feature_log_prob_[1]

    # Generate two dict: word:log_prob
    negative_word_log_prob_dict = {}
    positive_word_log_prob_dict = {}
    for word, index in vec.vocabulary_.items():
        negative_word_log_prob_dict[word] = negative_log_prob[index]
        positive_word_log_prob_dict[word] = positive_log_prob[index]
    if val_reviews is None:
        return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_test_reviews, vec
    else:
        return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_val_reviews, transformed_test_reviews, vec 

In [197]:
def calculate_reweight_dict(pos, neg, dataset, threshold=0.1):
    wrongly_classified_dict = {} # counts how many times a token has negative impact on wrongly classified sentence
    token_dict = {} # counts how many times a token has appeared in total
    reweight_dict = {}
    for label, sentence in dataset:
        real_label = label
        tokens = sentence
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = 1
            else:
                token_dict[token] += 1
        if real_label == "positive": # marked as negative
            for token in tokens:
                if token not in pos: continue
                elif pos[token] < neg[token]:
                    if token not in wrongly_classified_dict:
                        wrongly_classified_dict[token] = 1
                    else:
                        wrongly_classified_dict[token] += 1
        elif real_label == "negative": # marked as positive
            for token in tokens:
                if token not in pos: continue
                elif neg[token] < pos[token]:
                    if token not in wrongly_classified_dict:
                        wrongly_classified_dict[token] = 1
                    else:
                        wrongly_classified_dict[token] += 1
    # print(wrongly_classified_token_dict)
    for word, prob in wrongly_classified_dict.items():
        # print(word, prob)
        weight = 0
        if pos[word] < neg[word]:
            weight = pos[word] - neg[word]
        elif neg[word] < pos[word]:
            weight = neg[word] - pos[word]
        multiplier = 1 - wrongly_classified_dict[word] / token_dict[word]
        if multiplier == 0:
            reweight_dict[word] = 0
        else:
            reweight_dict[word] = np.exp(weight) * multiplier
            # print("{}:{} {}".format(word, weight, multiplier))
            # print("{}:{} -> {}".format(word, pos[word] - neg[word], reweight_dict[word]*pos[word] - reweight_dict[word]*neg[word]))
    return reweight_dict

In [198]:
'''
This method will take in a word:scale dict, then take in the negative and positive word:log_probability dict, manually change the weight of the words in the model and the dict
Input: word_change_scale: this is the word-scale dictionary, how much the weight of the word should be changed, For example if the value is 0.5, we will say 
                          the probability of the word in negative class should multiply 0.5, in original probability, we take power to the scale
       model: the trained naive bayes model, which the feature_log_prob_ attribute will be manually changed based on previous two params
       negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class, which some values will be changed
       positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class, which some values will be changed
       vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
return: negative_word_change_scale: The modified negative dict
        positive_word_change_scale: The modified positive dict
        model: The modified naive bayes model
'''

def change_weight(word_change_scale, model, negative_word_log_prob_dict, positive_word_log_prob_dict, vec):
    for word, scale in word_change_scale.items():
        # change the weight of words in negative and positive word:log_prob dict
        negative_word_log_prob_dict[word] *= scale
        positive_word_log_prob_dict[word] *= scale

        # change the weight of words in the model
        index_in_model = vec.vocabulary_[word]
        model.feature_log_prob_[0][index_in_model] *= scale
        model.feature_log_prob_[1][index_in_model] *= scale

    return negative_word_log_prob_dict, positive_word_log_prob_dict, model

### Bag of Word - Naive Bayes

In [199]:
neg, pos, mnb, transformed_val_reviews, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, test_reviews, val_reviews=val_reviews, alpha = 1)

In [200]:
unseen_neg = mnb.feature_log_prob_[0].min()
unseen_pos = mnb.feature_log_prob_[1].min()
print(unseen_neg, unseen_pos)

-15.032443811652895 -15.049387153559886


In [205]:
reweight_dict = {}

In [206]:
val_pred, val_acc = naive_inference(reweight_dict, pos, neg, unseen_neg, unseen_pos, val_reviews, val_sentiments)
test_pred, test_acc = naive_inference(reweight_dict, pos, neg, unseen_neg, unseen_pos, test_reviews, test_sentiments)
print(f"val accuracy is {val_acc}, test accuracy is {test_acc}")


0.8441
0.8472
val accuracy is 0.8441, test accuracy is 0.8472


In [208]:
wrong_labeled = []
pattern = r'[^A-Za-z0-9]+'

for i in range(len(val_reviews)):
    start_index = 30000
    if(val_pred[i] != val_sentiments[i]):
        sentiment = val_sentiments[i]
        review = str(imdb_data['review'][start_index + i])
        review = re.sub(pattern, " ", review.lower()).split()
        # review = test_reviews[i].strip().lower().split()
        wrong_labeled.append((sentiment, review))

reweight_dict = calculate_reweight_dict(pos, neg, wrong_labeled)
# neg, pos, mnb = change_weight(reweight_dict, mnb, neg, pos, vec)

In [209]:
val_pred_2, val_acc_2 = naive_inference(reweight_dict, pos, neg, unseen_neg, unseen_pos, val_reviews, val_sentiments)
test_pred_2, test_acc_2 = naive_inference(reweight_dict, pos, neg, unseen_neg, unseen_pos, test_reviews, test_sentiments)
print(f"val accuracy is {val_acc_2}, test accuracy is {test_acc_2}")

# for i in range(len(val_pred)):
#     if val_pred[i] != val_pred_2[i]:
#         tokens = str(imdb_data['review'][start_index + i])
#         tokens = re.sub(pattern, " ", review.lower()).split()
#         for token in tokens:
#             weight = 1
#             pprob = unseen_pos
#             nprob = unseen_neg
#             if word in reweight:
#                 weight = reweight[word]
#             if word in pos:
#                 pprob = pos[word]
#             if word in neg:
#                 nprob = neg[word]
#             final_result += weight*pprob - weight*nprob



0.9031
0.841
val accuracy is 0.9031, test accuracy is 0.841


In [210]:
def naive_inference(reweight, pos, neg, unseen_neg, unseen_pos, test_reviews, test_sentiments):
    correct = 0
    prediction = []
    for i in range(len(test_reviews)):
        word_list = test_reviews[i]
        word_list = re.sub(pattern, " ", word_list.lower()).split()
        # word_list = test_reviews[i].strip().lower().split()
        final_result = 0
        
        for word in word_list:
            weight = 1
            pprob = unseen_pos
            nprob = unseen_neg
            if word in reweight:
                weight = reweight[word]
            if word in pos:
                pprob = pos[word]
            if word in neg:
                nprob = neg[word]
            final_result += weight*pprob - weight*nprob
        if final_result > 0:
            prediction.append("positive")
        elif final_result < 0:
            prediction.append("negative")
        if (final_result > 0 and test_sentiments[i] == "positive") or (final_result < 0 and test_sentiments[i] == "negative"):
            correct += 1
    print(correct/len(test_reviews))
    return prediction, correct/len(test_reviews)

In [211]:
for i in range(10000):
    start_index = 40000
    if(test_pred[i] != test_sentiments[i]):
        sentiment = str(imdb_data['sentiment'][start_index + i])
        review = str(imdb_data['review'][start_index + i])
        review = re.sub(pattern, " ", review.lower())
        wrong_labeled.append((sentiment, review))

reweight_dict = calculate_reweight_dict(pos, neg, wrong_labeled)


## Reweight the model using perceptron

In [212]:
def perceptron_update(positive_prob_dict, negative_prob_dict, unseen_pos, unseen_neg, val_reviews, val_sentiments, test_reviews, test_sentiments, max_iter=10, learning_rate=0.02):
    """
    need to handle unseen word here
    has to be a method
    """
    initial_weight = {}
    for i in range(max_iter):
        random_array = np.arange(len(val_sentiments))
        np.random.shuffle(random_array)
        correct = 0
        for index in random_array:
            word_list = val_reviews[index]
            word_list = re.sub(pattern, " ", word_list.lower()).split()
            # word_list = val_reviews[index].strip().lower().split()
            final_result = 0
            for word in word_list:
                weight = 1
                pprob = unseen_pos
                nprob = unseen_neg
                if word not in initial_weight:
                    initial_weight[word] = 1
                if word in initial_weight:
                    weight = initial_weight[word]
                if word in positive_prob_dict:
                    pprob = positive_prob_dict[word]
                if word in negative_prob_dict:
                    nprob = negative_prob_dict[word]
                final_result += weight*pprob - weight*nprob
            if (final_result > 0 and val_sentiments[index] == "positive") or (final_result < 0 and val_sentiments[index] == "negative"):
                correct += 1
                for word in word_list:
                    initial_weight[word] -= learning_rate/len(val_sentiments)*initial_weight[word]
            else:
                for word in word_list:
                    pprob = unseen_pos
                    nprob = unseen_neg
                    if word in positive_prob_dict:
                        pprob = positive_prob_dict[word]
                    if word in negative_prob_dict:
                        nprob = negative_prob_dict[word]
                    if val_sentiments[index] == "positive":
                        initial_weight[word] += learning_rate*((pprob-nprob) - 1/len(val_sentiments)*initial_weight[word])
                    else:
                        initial_weight[word] += learning_rate*((nprob-pprob) - 1/len(val_sentiments)*initial_weight[word])
        test_pred, test_acc = naive_inference(initial_weight, positive_prob_dict, negative_prob_dict, unseen_neg, unseen_pos, test_reviews, test_sentiments)
        print(f"finish training epoch {i}, the val accuracy is {correct/len(val_sentiments)}, the test accuracy is {test_acc}")
        
    return initial_weight

In [213]:
reweight = perceptron_update(pos, neg, unseen_pos, unseen_neg, val_reviews, val_sentiments, test_reviews, test_sentiments)

0.8734
finish training epoch 0, the val accuracy is 0.8635, the test accuracy is 0.8734
0.8817
finish training epoch 1, the val accuracy is 0.881, the test accuracy is 0.8817
0.8841
finish training epoch 2, the val accuracy is 0.8939, the test accuracy is 0.8841
0.8848
finish training epoch 3, the val accuracy is 0.9042, the test accuracy is 0.8848
0.8855
finish training epoch 4, the val accuracy is 0.9119, the test accuracy is 0.8855
0.8858
finish training epoch 5, the val accuracy is 0.9167, the test accuracy is 0.8858
0.8846
finish training epoch 6, the val accuracy is 0.9243, the test accuracy is 0.8846
0.8855
finish training epoch 7, the val accuracy is 0.93, the test accuracy is 0.8855
0.8861
finish training epoch 8, the val accuracy is 0.9354, the test accuracy is 0.8861
0.8843
finish training epoch 9, the val accuracy is 0.9434, the test accuracy is 0.8843


In [15]:
weight_printed_version = dict(sorted(reweight.items(), key=lambda item: item[1]))
with open("weight_dict.txt", "w") as f:
    for k, v in weight_printed_version.items():
        f.write(str(k))
        f.write(": ")
        f.write(str(v))
        f.write('\n')


In [16]:
import pickle

with open('weight.pickle', 'wb') as handle:
    pickle.dump(reweight, handle, protocol=pickle.HIGHEST_PROTOCOL)